# Installing and starting Oscar

First, you need to install the (compiler for) `julia`. For that, follow the installation 
instructions on the OSCAR website (https://oscar.computeralgebra.de) and make sure you 
install the latest julia version. 

Usually, you would then proceed to follow the installation instructions for 
OSCAR there, but for this course we will need a modified version of OSCAR which comes with some 
extra patches that have not (yet) made their way in the official release. 
To install and use that version, open a julia REPL (Terminal, notebook, ...) and type

In [ ]:
using Pkg
]add https://github.com/HechtiDerLachs/Oscar.jl#CIMPA_school

Then you should be able to start OSCAR with the command 

In [ ]:
using Oscar

### Installing jupyter-notebook

What you see here is a so-called `jupyter` notebook. Those notebooks do not run out of 
the box with julia. On Ubuntu/Debian, you need to install the package `jupyter-notebook`. 
In addition, you need an extension of `julia` so that it can connect to `jupyter`. 
According to the OSCAR website, you are supposed to run

In [ ]:
using Pkg
Pkg.add("IJulia")

### Updating OSCAR during the course

It might be necessary during the course to update OSCAR. To do so, type

In [ ]:
] up

### Switching back to the official OSCAR version after the course

After the course is finished and you wish to continue using OSCAR, you can switch to the 
official release version as follows

In [ ]:
] rm Oscar
using Pkg
Pkg.add("Oscar")

# FIrst steps in OSCAR

## Polynomial rings, etc.

Some standard rings and fields are already there:

In [ ]:
QQ

In [ ]:
ZZ
# Try 2^1000 vs. ZZ(2)^1000

In [ ]:
GF(53)

Some others need to be manually created:

In [ ]:
R, (x,y,z) = QQ["x", "y", "z"]

Note that the above command returns a tuple `(R, v)` consisting of the polynomial ring `R` and another tuple `v = (x,y,z)` containing its variables.

We can now investigate both `R` and the variables:

In [ ]:
R

In [ ]:
x^2 + 3*x

This does not tell us much. But of course, there is more 
going on in the background. For instance, everything in 
julia has a type and so does our variable `x`. We will 
come back to that later.

In [ ]:
typeof(x)

We can also access the variables via the `getindex`-function 
for the ring `R`:

In [ ]:
x == R[1]

Any ring and its elements in Oscar have a parent-element relationship. We can check which ring an element belongs to:

In [ ]:
parent(x^2 + 4*x^3+ y)

In [ ]:
parent(x) == R

Let us create another ring over a different field. 
This time, we use another constructor:

In [ ]:
S, w = PolynomialRing(GF(101), "w" => 1:5)

In [ ]:
w[3]

In [ ]:
parent(w[2]) == R

In [ ]:
parent(w[2]) == S

## Matrices, ideals, and modules

### Matrices over polynomial rings

Given a ring `R`, we can set up matrices over it:

In [ ]:
M = R[x y z; z x y^2]

Note that the separation of a rows entries is done by whitespaces. 
Unfortunately, that means that you have to be careful when using
blanks in your expressions:

In [ ]:
M = R[x, y z; z x y^2] #TODO: Modify and play around to create a mess

Especially, there is some potential of conflict with the `getindex` method when creating `1x1`-matrices:

In [ ]:
f = R[2]^2 # the first variable of `R` squared

In [ ]:
F = R[2;]^2 # the 1x1-matrix A = (2) squared

In [ ]:
M[1, 2] # the (1,2)-th entry of the matrix M

Matrices can further be processed; for instance, we can take minors:

In [ ]:
minors(M, 2) # return a list of 2x2-minors of M

...or submatrices:

In [ ]:
subM1 = M[1:end, 1:2] # this takes two `range`s as input

In [ ]:
subM2 = M[1:2, [1,3]] # this takes a `range` and a `Vector{Int}` as input

If we want a specific minor of a matrix, we have to specify the 
corresponding square submatrix first and then take the determinant:

In [ ]:
det(subM2)

### Ideals in polynomial rings

We can create ideals in polynomial rings like `R` from a list of generators as follows:

In [ ]:
I = ideal(R, [x^2 - y*z, x*y^2 - z^2, -x*z + y^3])

Any ideal can be asked for the generators with which it was once constructed:

In [ ]:
g = gens(I)

We can test ideal membership in `I` for elements of `R`:

In [ ]:
x + 1 in I

In [ ]:
3*(x^2 - y*z) + z*(x*y^2 - z^2) in I

Moreover, for any given element $f \in I$, we can ask 
for the coefficients of $f$ in a linear combination of the generators of $I$:

In [ ]:
coordinates(3*(x^2 - y*z) + z*(x*y^2 - z^2), I)

Note that such "coordinates" are not unique!

## Geometry: Varieties and their singular loci

For the computer, an affine variety is usually modeled by the ideal $I \subset R$ defining it. 
In Oscar, we even have data types for varieties (later!), but we will stick with the algebraic 
side for the moment, thinking of it geometrically. 

Let us come back to one of our previous examples...

In [ ]:
R, (x,y) = QQ["x", "y"]
f = x*y*(1-x-y)
I = ideal(R, f)

... and let us find out 
what we can about the underlying variety.

In [ ]:
dim(I)

So the variety defined by `I` is an algebraic curve. Does it have singular points? Let's apply the jacobian criterion!

In [ ]:
g = gens(I)
Dg = jacobi_matrix(g)

In [ ]:
J = ideal(R, minors(Dg, 1)) + I # Apply the jacobian criterion

Often, the output is not very readable. Since we are only 
interested in the geometric locus (and not the scheme theoretic 
structure), there is some hope that the radical is more simple.
But be warned! Taking the radical is in general a very 
expensive procedure. 

In [ ]:
radical(J)

No luck in this case. As a next step, let us try to decompose `J` using the `primary_decomposition`:

In [ ]:
primary_decomposition(J)

We can wrap this in a new function:

In [ ]:
function radical_of_singular_locus(I) 
    d = dim(I)                     # the dimension of the ideal 
    R = base_ring(I)               # the ring in which `I` lives
    g = gens(I)
    Dg = jacobi_matrix(g)
    n = ngens(R)                   # the number of variables of `R`
    Ising = I + ideal(R, minors(Dg, n-d))
    return radical(Ising)
end

Let's apply it to our previous ideal and see whether it works.

In [ ]:
radical_of_singular_locus(I)

## Local rings in OSCAR

In Oscar, we can localize polynomial rings at geometric points $p \in \mathbf A^n$. This 
is important in singularity theory, since geometrically the algebraic procedure of localization 
corresponds to passing from an affine variety to its germ at a point.

In [ ]:
R, (x,y,z) = QQ["x", "y", "z"]
U = complement_of_ideal(R, [0,0,0]) # The multiplicative set of functions not vanishing at the origin
L, map_from_R_to_L = Localization(R, U)

Just like, for example, `Int` is casted to `Float` when evaluating `1 + 1.3`, 
elements of the original ring `R` are automatically casted to 
elements of `L` whenever the context requires it; we can freely 
do arithmetic mixing elements of both rings.

In [ ]:
a = x + inv(L(y-1))

In [ ]:
parent(a) == L

Also, mathematical comparison works as expected:

In [ ]:
one(L) == one(R)

For comparison of the objects in the computer (i.e. in memory), there is the `===` operator. It gives another result:

In [ ]:
one(L) === one(R)

Let us come back to geometry. 
We set up the following variety. X is the union of a 
hyperplane H = {z = 1} and the cylinder over a smooth
conic C = {x(x-2)-y^2 = 0}. Both components are smooth, 
but their union is singular along the intersection of 
both components.

In [ ]:
h = z-1
f = x*(x-2)-y^2
I = ideal(R, f*h)

Let us use our above function to find out about the 
singular locus:

In [ ]:
J = radical_of_singular_locus(I)

In [ ]:
J_loc = L(J)

`J` is now an ideal in the ring `L`. 

In [ ]:
base_ring(J_loc) == L

In particular, it has another data type working in the background!

In [ ]:
typeof(J_loc)

In [ ]:
typeof(J)

Nevertheless, `J_loc` should behave just like any ideal should behave, i.e. 
one should be able to ask for a set of generators, one should be able 
to test ideal membership of elements, etc. Let us check this in our 
geometric example.

Locally at the point p = (0,0,0), we only have the 
smooth component C and its singular intersection 
with H is far away. Hence, the localized ideal of 
the singular locus should be trivial. Indeed, 
we find:

In [ ]:
one(L) in J_loc

We can also try to use our above method on the localization of `I` directly:

In [ ]:
I_loc = L(I)

In [ ]:
radical_of_singular_locus(I_loc) 